# **PCA**

**Principal Component Analysis (PCA)** is a dimensionality reduction technique widely
used in data analysis and machine learning. 

Its primary goal is to transform high-dimensional data into a lower-dimensional representation, capturing the most important
information.

*HOW DO YOU DO A PRINCIPAL COMPONENT ANALYSIS?*

1. Standardize the range of continuous initial variables
2. Compute the covariance matrix to identify correlations
3. Compute the eigenvectors and eigenvalues of the covariance matrix to identify the principal components
4. Create a feature vector to decide which principal components to keep
5. Recast the data along the principal components axes

**Principal components** are new variables that are constructed as linear combinations or mixtures of the initial variables. These combinations are done in such a way that the new variables are uncorrelated and most of the information within the initial variables is squeezed or compressed into the first components. 

<div style="text-align:center;">
    <img src="pca.png" alt="Example Image" width="500">
</div>

### *Advantages*

1. **Dimensionality reduction:** PCA effectively reduces the number of features, which is beneficial for models that
suffer from the curse of dimensionality.

2. **Feature independence:** Principal components are orthogonal (uncorrelated), meaning they capture independent information, simplifying the interpretation of the reduced features.

3. **Noise reduction:** PCA can help reduce noise by focusing on the components that explain the most
significant variance in the data.

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widget
import plotly.graph_objs as go
from PIL import Image
from sklearn.metrics import mean_squared_error as mse
from ipywidgets import interact, fixed
import cv2 as cv


### **Step 1:** Data loading

Firstly, we need to dowload our data and prepare it for further manipulations.

In this part we read grayscale images from our dataset, resize them to the same size (100x100 pixels), flattens them, and stores them as column vectors in a NumPy array.

In [4]:
path = '../brain_aneurysm_dataset/no'

A = []
A_flatten = []

size = 100

for img in os.listdir(path):

    image = cv.imread(os.path.join(path, img), cv.IMREAD_GRAYSCALE)
    
    # convert to matrix
    matrix = Image.fromarray(image)

    # resize
    resize_img = np.array(matrix.resize((size, size)))

    A.append(resize_img)

    # flatten
    image_flatten = resize_img.flatten()

    A_flatten.append(image_flatten)

# numpy array
A_flatten = np.asarray(A_flatten)

# transposed
A_flatten = A_flatten.T

In [5]:
A_flatten[0]

array([  1,  16,   3,   0,   0,   0,   4,   0,   0,   0,  27,   0,   0,
         0,   7,   0,   0,  10,   0,   0,  99,   0,   3,   0,   2,  45,
         0,   0,   0,   0,  11,  10,   0,  67,   0,   0,  16,   2,  77,
         0,   0,   1,   0,  23,   0, 252,   2,   0,   3,   1,   0,   4,
         3,   9,   0,  26,   0,   0,   0,   0,   2,   3,  12,   0,  35,
         7,   0,   2,   1,   0], dtype=uint8)

### **Step 2:** Data normalization

PCA is sensitive to the scale of the data. Standardizing the features ensures that each feature contributes equally to the analysis. It involves scaling the features to have a *mean* of 0 and a *standard deviation* of 1.

By subtracting the mean of each feature from the dataset it centers the data around the origin. This step ensures that the first principal component captures the direction of maximum variance.

This process ensures that the variables are standardized, allowing for a fair and unbiased comparison among them.

In [6]:
def normalization(A):

    # column mean and standard deviation
    mu = np.mean(A, axis = 0)
    std = np.std(A, axis = 0)  
    
    # substract the mean from A
    A_centered = A - mu   
    
    # zero devizion
    std_filled = std.copy()
    std_filled[std == 0] = 1
    
    # normalized A
    A_norm = A_centered / std_filled
    
    return A_norm, mu, std

In [7]:
A_norm, mu, std = normalization(A_flatten)
A_norm[0], mu[0], std[0]

(array([-0.75057428, -0.78146657, -0.81806781, -0.84981493, -0.78832268,
        -0.73458662, -0.78546455, -0.85359589, -1.03930308, -0.86119516,
        -1.32129468, -0.85418103, -0.63179615, -1.09521945, -1.01184732,
        -1.01479543, -0.81366399, -0.76981093, -0.98875834, -0.76674641,
         0.98719965, -0.8525469 , -1.01760369, -0.55204478, -1.05332863,
         0.23295598, -1.16999486, -1.09591614, -0.99099847, -0.66535457,
        -1.2074218 , -0.76927773, -0.85359589,  0.67699491, -1.16999486,
        -0.79636922, -0.77118396, -1.05332863,  0.83283191, -1.12230673,
        -0.64567761, -0.88807423, -0.96451991, -1.00437391, -1.08807415,
         2.22382554, -1.09458609, -1.01475946, -1.02564313, -0.95476697,
        -0.65226068, -1.1859063 , -1.01760369, -1.18466176, -0.65079856,
        -0.90420934, -0.65226068, -0.89889429, -0.76399282, -0.7736369 ,
        -1.30896572, -1.02564313, -0.98577267, -0.85418103, -0.72556173,
        -1.11364224, -0.8525469 , -1.30896572, -0.6

### **Step 3:** Computing the covariance matrix

The covariance matrix summarizes the pairwise covariances between the different features in the dataset. It's a square matrix where each element represents the covariance between two features.

This matrix helps us to understand how the features are related to each other. If certain features are highly correlated, it suggests that they provide similar information. By examining the covariance matrix, we can determine which features to remove in order to eliminate redundancy.

Mathematically, the covariance between two variables $X_i$ and $X_j$ is calculated as:

$$Cov(X_i, X_j) = \frac{1}{n-1} \sum_{k=1}^{n} (x_{ik} - \bar{x}_i)(x_{jk} - \bar{x}_j)$$



In [8]:
def covariance(A):

    # covariance matrix S
    S = 1 / (len(A) - 1) * np.matmul(A, A.T)
    
    return S

In [9]:
A_covariance = covariance(A_norm)
A_covariance[0]

array([0.00637285, 0.00603807, 0.00594348, ..., 0.00598129, 0.00609126,
       0.00655558])

### **Step 4:** Computing eigenvectors and eigenvalues of covariance matrix

Eigenvectors are the directions along which the data varies the most. They represent the principal components of the data.
As the covariance matrix is symmetric all eigenvectors are pairwise orthogonal.

Eigenvalues indicate the variance of the data along the corresponding eigenvectors. Larger eigenvalues correspond to directions of greater variance.

The eigenvector that corresponds to the largest eigenvalue identifies the direction of maximum variance, which is considered as the *first principal component*, second largest variance represents the *second principal component*, and so on.

The eigenvectors and eigenvalues are typically computed by performing eigendecomposition on the covariance matrix.

In [10]:
def eigenvalues_vectors(A):

    # eigenvalues and eigenvectors
    EV, EVc = np.linalg.eigh(A)  

    # desceding order
    order = np.argsort(-EV)
    EV = EV[order]
    EVc = EVc[:, order]
    
    return EV, EVc

### **Step 5:** Calculating the projection matrix

This step involves transforming the original data onto the new subspace spanned by the selected principal components.

The projection is done by multiplying the original data matrix by the matrix of selected eigenvectors.

The projected data represents the dataset in the reduced-dimensional space defined by the principal components.

In [11]:
def PCA(A, components):
    
    S = covariance(A)
    
    EV, EVc = eigenvalues_vectors(S)
    U = EVc[:, range(components)]
    
    # projection matrix which projects our input data onto vector space spanned by the eigenvectors
    P = np.matmul(U, U.T)
    
    return P

### **Step 6:** Calculating loss

In this part, we calculate the reconstruction error between the original data and the transformed data for each number of components.

The goal is to evaluate how well the dataset can be represented and reconstructed in a lower-dimensional space using different numbers of principal components, and to analyze how the reconstruction error changes as the number of components varies.

In [12]:
losage = []
transformation = []

component_max_number = len(np.array(A_flatten).T)

for component in range(1, component_max_number + 1):

    P = PCA(A_norm.T, component)
    y_T = np.matmul(A_flatten, P)
    error = mse(y_T, A_flatten)
    transformation.append(y_T)
    losage.append((component, error))


### **Step 7:** Number of principal components analysis

In this part of the code we aim to provide an interactive way to visualize the reconstructed images by adjusting the number of principal components used in the PCA reconstruction process.

As we can observe, the more principal components we choose, the better quality our image has.

In [13]:
transformation = np.asarray(transformation)
losage = np.asarray(losage)

In [14]:
@interact(components=widget.IntSlider(min = 1, max = component_max_number - 1, value=10), image=fixed(image))

def display_pca_image(components, image):
    plt.imshow(transformation[components, : , 0].reshape(100, 100), cmap='gray')
    plt.show()
    

interactive(children=(IntSlider(value=10, description='components', max=69, min=1), Output()), _dom_classes=('…

### **Step 8:** Visualization

Now, let's visualize how MSE changes depending on number of principal components.

In [15]:
trace = go.Scatter(x = losage[:, 0], y = losage[:, 1], mode='markers', marker=dict(color='red'))
fig = go.Figure(trace)
fig.update_layout(title="Dependence of MSE and number of principal components",
                  xaxis_title="Number of principal components",
                  yaxis_title="MSE")
fig.show()
